## Imports

In [1]:
# imports - matploltib, pandas, NumPy
import pandas as pd 

In [2]:

import datetime as dt
from datetime import timedelta

import folium

import warnings
warnings.filterwarnings('ignore')


In [3]:
# imports - plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from IPython.display import HTML

from plotly.offline import iplot
import plotly as py
import cufflinks as cf

py.offline.init_notebook_mode(connected=True)
cf.go_offline()


***
## Load the dataset

In [4]:
df = pd.read_csv('citi_bike_2016.csv')
df.head()

,id,gender_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,25242,2,2016-05-30 15:23:05,2016-05-30 15:52:59,-73.976485,40.759923,-74.002150,40.730386,1794
1,20900,1,2016-04-24 19:56:47,2016-04-24 20:02:17,-74.003664,40.743174,-74.002150,40.730386,329
2,18792,1,2016-06-25 15:08:38,2016-06-25 15:15:57,-73.991908,40.716059,-74.005524,40.711464,438
3,17420,1,2016-06-28 18:14:17,2016-06-28 18:35:25,-73.986569,40.701485,-73.989900,40.714275,1268
4,22403,1,2016-06-11 17:10:33,2016-06-11 17:14:44,-73.989551,40.740343,-73.990093,40.737050,251


In [5]:
print(f"""
Shape:
{df.shape}
------------------------
Null values:
{df.isnull().sum()}
------------------------
Duplicates:
{df.duplicated('id').sum()}
""")


Shape:
(4500000, 9)
------------------------
Null values:
id                   0
gender_id            0
pickup_datetime      0
dropoff_datetime     0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
trip_duration        0
dtype: int64
------------------------
Duplicates:
4490842



We can see there are a lot of duplicate values on ID, but they are not presenting a trips lats and longs.<br>
Therefore, we will drop this column as it is not required.

In [6]:
data = df.drop(['id'], axis = 1)
# data.head(1)

In [7]:
from pyproj import crs
import geopandas as gpd
import matplotlib.pyplot as plt

### Creation of map

Simple Creation of a map

In [8]:
# Create a Map instance
m = folium.Map(location=[60.25, 24.8], zoom_start=10, control_scale=True)
# m

In [9]:
#  you can also save it to html file
outfp = "base_map.html"
m.save(outfp)

***
### Change of canvas

In [10]:
# Let's change the basemap style to 'Stamen Toner'
m = folium.Map(location=[40.730610, -73.935242], tiles='Stamen Toner',
                zoom_start=12, control_scale=True, prefer_canvas=True)

# m

In [11]:
# Also change of colors
m = folium.Map(location=[41,29],
tiles="Stamen Watercolor",
 width="%100",
height="%100",
zoom_start=10)

***
### Adding layers and markers

In [12]:
# Create a Map instance
m = folium.Map(location=[60.20, 24.96],
    zoom_start=12, control_scale=True)

# Add marker
# Run: help(folium.Icon) for more info about icons
folium.Marker(
    location=[60.20426, 24.96179],
    popup='Kumpula Campus',
    icon=folium.Icon(color='green', icon='ok-sign'),
).add_to(m)

#Show map
# m

In [27]:
# Add Multiple markers
map = folium.Map(location=[data.pickup_latitude.mean(),
                           data.pickup_longitude.mean()], zoom_start=14, control_scale=True)
for index, location_info in data.head(20).iterrows():
    folium.Marker([location_info["pickup_latitude"], location_info["pickup_longitude"]], 
                  popup=location_info["pickup_datetime"]).add_to(map)    
# map

Add A circle to the point for more visualize

In [30]:
# circle the point
m = folium.Map(location=[41,29],tiles="Stamen watercolor",
width="%100",
height="%100",
zoom_start=10)
folium.Circle(
radius=5000,
location=[41,29],
color='crimson',
fill=False,).add_to(m)
# m

Marker + Circle of the point

In [32]:
m = folium.Map(location=[40.75834,30.38001])
folium.Marker(location=[40.75834,30.38001], popup =  'Sakarya').add_to(m)
folium.CircleMarker(location=(40.75834,30.38001),radius=100, fill_color='blue', fill=True).add_to(m)
# m

***
### Adding images

In [34]:
import folium
from folium.plugins import FloatImage


url = (
    "https://raw.githubusercontent.com/ocefpaf/secoora_assets_map/"
    "a250729bbcf2ddd12f46912d36c33f7539131bec/secoora_icons/rose.png"
)

m = folium.Map([-13, -38.15], zoom_start=10)

FloatImage(url, bottom=40, left=65).add_to(m)

# m

***
### Heatmaps

In [17]:
# Create a list of coordinate pairs
locations = list(zip(data["pickup_latitude"].head(1000), data["pickup_longitude"].head(1000)))

In [18]:
locations[0]

(40.75992262, -73.97648516)

In [36]:
from folium.plugins import HeatMap

# Create a Map instance
m = folium.Map(location=[40.75992262, -73.97648516], tiles = 'stamentoner', zoom_start=12, control_scale=True)

# Add heatmap to map instance
# Available parameters: HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, 
# max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True)
HeatMap(locations).add_to(m)

# Show map
# m

***
### Clustered point map

In [20]:
from folium.plugins import MarkerCluster

In [21]:
# Create a list of coordinate pairs
locations = list(zip(data["pickup_latitude"].head(1000), data["pickup_longitude"].head(1000)))

In [22]:
m = folium.Map(location=[40.75992262, -73.97648516], tiles = 'cartodbpositron', zoom_start=11, control_scale=True)

In [38]:
# Create a folium marker cluster
marker_cluster = MarkerCluster(locations)

# Add marker cluster to map
marker_cluster.add_to(m)

# Show map
# m

***
### Create routes

In [24]:
locations = list(zip(data["pickup_latitude"].head(5), data["pickup_longitude"].head(5)))

In [39]:
import folium

m = folium.Map(location=[40.75992262, -73.97648516],
              zoom_start=12)

loc = locations

folium.PolyLine(loc,
                color='blue',
                weight=5,
                opacity=0.8).add_to(m)
# m